In [1]:
# Imports
import gradio as gr
import g4f
import tiktoken

New pypi version: 0.1.6.6 (current: 0.1.6.2) | pip install -U g4f


In [28]:
# Count tokens
system_message = [{"role": "system", "content": "Ты пират, отвечай как пират."}]
max_response_tokens = 1024
token_limit = 16000
last_message = ""
def num_tokens_from_messages(messages):
    encoding= tiktoken.get_encoding("cl100k_base")  #model to encoding mapping https://github.com/openai/tiktoken/blob/main/tiktoken/model.py
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

In [3]:
# Convert history to openai format
def gradio_to_openai_history(history):
    result = []
    for dialogue in history:
        result.append({"role": "user", "content": dialogue[0]})
        result.append({"role": "assistant", "content": dialogue[1]})
    return result

In [29]:
# Chat response
def response(message, history):
    conversation = system_message + gradio_to_openai_history(history) + [{"role": "user", "content": message}]
    conv_history_tokens = num_tokens_from_messages(conversation)

    while conv_history_tokens + max_response_tokens >= token_limit:
        del conversation[1] 
        conv_history_tokens = num_tokens_from_messages(conversation)
    
    last_message = g4f.ChatCompletion.create(
        model="gpt-3.5-turbo-16k-0613",
        provider=g4f.Provider.NeuroGPT,
        messages=conversation,
        stream=False,
        )
    return last_message

In [32]:
def find_item():
    return g4f.ChatCompletion.create(
        model="gpt-3.5-turbo-16k-0613",
        provider=g4f.Provider.NeuroGPT,
        messages=[{"role": "system", "content": "Проанализируй данный текст и найди в нем предмет. Если здесь нет предмета, то напиши 'нет'."},
                  {"role": "user", "content": last_message}],
        stream=False,
        )

In [33]:
with gr.Blocks() as demo:
    chatbot = gr.ChatInterface(response)
    find = gr.Button("Find")
    find.click(find_item, outputs=gr.Text(), show_progress=True, status_tracker=None)
demo.launch()

Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\KorolOrol\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "c:\Users\KorolOrol\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\util\connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "c:\Users\KorolOrol\AppData\Local\Programs\Python\Python310\lib\socket.py", line 955, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11001] getaddrinfo failed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\KorolOrol\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py", line 790, in urlopen
    response = self._make_request(
  File "c:\Users\KorolOrol\AppData\Local\Programs\Python\Python3

In [34]:
print(last_message)